In [ ]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

## Definitions

|var name|value<br>exp1|value<br>exp2|unit|definition|
|---|---|---|---|---|
|posrlon|2.54517|same|[°E]||
|posrlat|2.54879|same|[°N]|[coordinats](https://xkcd.com/2170/) of the emission point<br>(here in rotated coordinats of the forecast model)|
|dt|30|same|[s]|model time step|
|scale|1.e-6|same|-|scaling between measured particles and model particles<br>scale=1 model particle = 1 million measured particles|
|emission time|10:50-11:45|13:40-14:30|-|time with particle emission|
|ntmsp|110|100|-| number of time steps with particle emission|
|itime|20.85|23.65|[h]| start time of the emission in hours after model start|
|velox|0.|same|[ms-1]|the velocity of moving source in x direction|
|veloy|0.|same|[ms-1]|the velocity of moving source in y direction|
|ofilename|exp1|exp2|-|name of output file|
|plumeheight|5.|same|[m]|height of the dust plume|
|plumeradius|5.|same|[m]|radius of the dust plume|
|ngridpoints|100|same|-|number of horizontal gridpoints of the plume|




In [ ]:
posrlon=2.54517
posrlat=2.54879

dt=30

scale=1.e-6

ntmsp = 100

itime=23.65

velox=0.
veloy=0.

ofilename = 'exp2'
ofilename+= '_'+str(scale)+'.out'

plumeheight=5.
plumeradius=5.

ngridpoints=100
dx=2.*plumeradius/ngridpoints # spacing of the gridboxes
dz=dx 

# I divide the area form -plumeradius to +plumeradius in x and y direction 
# and from 0 to +plumeheight in z direction in grid boxes with the size dx**3
ngridbox=int(ngridpoints**2*plumeheight/dz) 

## Input
### Produce random data for testing

In [ ]:
# Todo

### Read in data from Environmental Dust Monitor

In [ ]:
# filenames
fname1 = 'GRIMM_1.5m_diff_exp2.data'
fname2 = 'GRIMM_3.8m_diff_exp2.data'

f1 = open(fname1,encoding = 'unicode_escape')
f2 = open(fname2,encoding = 'unicode_escape')

# read header
head=f1.readline()
head=f1.readline()
head=f2.readline()
head=f2.readline()

# read bin names form the header
binnames = []
for i in head.split():
    if(int(i[0].isdigit())):
        binnames.append(i)
binnames.append('>32.0')

# read the data to np.array
data1 = np.loadtxt(f1,usecols=range(2,33))
data2 = np.loadtxt(f2,usecols=range(2,33))

#dimension of the data set
dim_data=data1.shape


# definition of the peak in the data & removal of the background noise

# exp1 
# dat1 = data1[459,:]-np.median(data1[:,:],axis=0)
# dat2 = data2[463,:]-np.median(data2[:,:],axis=0)

# exp2
dat1 = data1[181,:]-np.median(data1[:,:],axis=0)
dat2 = data2[179,:]-np.median(data2[:,:],axis=0)

## Create the Bubble

In this section, I use the EDM data to create a vertical profile of the particle concentration. 
From there I project the profile to a half-sphere to get a concentration field.
Then I calculate how many particles there should be in every grid box of the sphere. 

In [ ]:
# this lists store the data for the plotting later
xp_plt=[]
yp_plt=[]
contour_plt=[]

# loop over all bins
for ibin in range(len(dat1)):   # range(11,23) -> particles between 1 and 10 µm
    print(ibin)
    print(binnames[ibin])
    
    # negative particle concentrations may occur in some bins because of the background noise removal
    # filter them out
    if dat1[ibin] <= 0:
        dat1[ibin] = 0.
        
    if dat2[ibin] <= 0:
        dat2[ibin] = 0.
        
        
    # define the data points we are using: surface:dat1, 1.5m:dat1, 3,8m:dat2, top of plume:0 
    # it is defined as x points because we need a funktion y(x) for the fitting 
    # it is flipped later 
    xpoints=[dat1[ibin],dat1[ibin],dat2[ibin],0]
    ypoints=[0.,1.5,3.8,plumeheight]



    # fitting with 5th order Poly (5th order seems to work well)
    x_p = np.linspace(0,plumeheight,ngridpoints/2)
    fit = np.polyfit(ypoints,xpoints,5)
    y_p = np.polyval(fit,x_p)
    
    # save data for plotting
    xp_plt.append(x_p)
    yp_plt.append(y_p)
    
    # the profile is now defined as funktion y_p(x_p). 
    # to make it a vertikal profile we flip it x_p(y_p)

    # the vertikal profile defines the particle conzentration 
    # in the center of the dust plume as a function of higth 
    
    # now we use the profile to create a 3D conzentration field in the shape of a half sphere
    # in the standart setting plumeradius=5. and plumehight=5.
    # the conzentration field is defined in a grid from -5 to 5 meter horizontal and 0 to 5 meter vertical. 
    # with ngridpoints=100 each gridbox has a size of 0.1**3 meter
    xg = np.linspace(-plumeradius, plumeradius,ngridpoints)
    yg = np.linspace(-plumeradius, plumeradius,ngridpoints)
    zg = np.linspace( 0.0, plumeheight,int(plumeheight/dz))
    
    print(zg.size)

    # get coordinates of the gridboxes with np.meshgrid
    X, Y, Z = np.meshgrid(xg, yg,zg)

    # a sphere is defined as x**2+y**2+z**2=r**2
    # this has to be normed by the plumeheight**2 
    # to get an conzentration field with 1 in the center and 0 at the edge
    # Then I multiply this with the vertical profile
    contour = (plumeheight**2-(X**2 + Y**2 + 1*Z**2))/plumeheight**2 * np.polyval(fit,Z)
    
    

    # the funktion above crate some artificials at the edges 
    # like "negativ" conzentrations outside of the bubble
    # and "shadow" bubbles in corners above the plume
    # The next few linse filter them out by setting everting to zero 
    # with is above the maximum profile heigth (note: not every profile reaches the possible plumeheight)
    
    zid=0
    while y_p[zid] <= 0:
        zid+=1
    while y_p[zid] > 0:
        zid+=1
        if zid == ngridpoints/2 -1:
            break
    contour[:,:,zid:]=0.0
    
    # if contour > 0. then contour == contour eles contour == 0.
    contour = np.where(contour > 0., contour,0.)
    
    # save this contour field for plotting 
    contour_plt.append(contour)
    


## Output 

### init the ofile

In [ ]:
# open new file
f=open(ofilename,'w')

# write header
f.write(ofilename+' \n')
hline='time'
hline+=5*' '+'lon'
hline+=12*' '+'lat'
hline+=12*' '+'height'
hline+=7*' '+'npart'
hline+=3*' '+'diam'
hline+=14*' '+'dens'
hline+=7*' '+'emission'
hline+='\n'
f.write(hline)
f.write(50*'-'+'\n')

# close file
f.close()

### Create particle startpoints

In [ ]:
    # ppgb - parts per grid box
    # Finaly the conzentration [parts per liter] in transformed into parts per gridbox.
    
#    ppgb = dx**3 * contour*1000 # size gridbox m^3 * part/liter * 1000 liter/m^3
#    nrmax=int(np.sum(ppgb)*scale)*ntmsp
#    ppgb = ma.masked_where(ppgb <= 0.,ppgb)

#

#    # ax3.voxels(X,Y,Z,contour)


#    nrmax=int(np.sum(ppgb)*scale)*len(rlons)

#    nrsum+=nrmax
#    points=np.zeros((nrmax,3))

#    print(nrmax)
#    ax3.set_xlabel('n='+str(nrmax),labelpad=-15.)



#    props=np.reshape(ppgb,ngridbox)/np.sum(ppgb)
#    npp=0
#    for pp in props.mask:
#        if pp:
#            props[npp]=0.0
#        npp+=1
#    if sum(props) == 0.:
#        print('fail')
#        print(ibin)
#        print(np.sum(ppgb))
#        print(ppgb)
#        plt.show()
#    randoms = np.random.choice(ngridbox, nrmax,p=props)

#    nr=0
#    for ran in randoms:
#        idx = np.unravel_index(np.ravel_multi_index((ran,), props.shape), ppgb.shape)
#        points[nr,0]=X[idx] + np.random.random()*dx-dx/2.
#        points[nr,1]=Y[idx] + np.random.random()*dx-dx/2.
#        points[nr,2]=Z[idx] + np.random.random()*dx-dx/2.
#        if points[nr,2] < 0:
#            points[nr,2]=0.1
#        nr+=1


#    # transform relative particle possitions to rotated coordinates
#    nrmax=int(nrmax/ntmsp)
#    for rlon,rlat,nr in zip(rlons,rlats,range(len(rlons))):
#        points[nr*nrmax:(nr+1)*nrmax,0]=rlon + meter2grad(points[nr*nrmax:(nr+1)*nrmax,0])
#        points[nr*nrmax:(nr+1)*nrmax,1]=rlat + meter2grad(points[nr*nrmax:(nr+1)*nrmax,1])


#    ax3.scatter(points[:,0],points[:,1],points[:,2],'b.',s=0.25)
#    # ax4.plot(points[:,0],points[:,1],'b.')
#    ax3.set_xticks([])
#    ax3.set_yticks([])

#    # ax3.legend(['n='+str(nrmax)])

#    x+=1
#    if x > 7:
#        x = 0
#        y+=1


#    pos=binnames[ibin].find('-')
#    if pos > 0:
#        Dmin=float(binnames[ibin][:pos])
#        Dmax=float(binnames[ibin][pos+1:])
#    else:
#        Dmin=float(binnames[ibin][1:])
#        Dmax=50.0
#    # print(Dmax-Dmin)
#    print()
#    # write points to new_start_file
#    npoint=0
#    #exp1 initime = 20.65
#    #exp2 initime = 23.50
#    #exp3 initime = 11.00
#    #exp4 initime = 15.00
#    time=0.0
#    initime=itime
#    for point in points:
#        npoint+=1
#        # create the output row
#        # time
#        if npoint> nrmax:
#            time+=0.0083
#            npoint=1

#        stime='{:8.4f}'.format(time+initime)
#        row=stime
#        # lon
#        nspace,ndigs,nfill=numspace(point[0])
#        row+=nspace*' '+str(point[0])[:ndigs]+nfill*'0'
#        #lat
#        nspace,ndigs,nfill=numspace(point[1])
#        row+=nspace*' '+str(point[1])[:ndigs]+nfill*'0'
#        # height
#        nspace,ndigs,nfill=numspace(point[2])
#        row+=nspace*' '+str(point[2])[:ndigs]+nfill*'0'
#        # npart
#        npart=1
#        nspace,ndigs,nfill=numspace(npart)
#        row+=nspace*' '+str(npart)
#        # Diam
#        diam=(np.random.random()*(Dmax-Dmin)+Dmin)*1e-6
#        nspace,ndigs,nfill=numspace(diam)
#        row+=nspace*' '+str(diam)[:ndigs]+str(diam)[-4:]
#        # Dens
#        dens=2650.0
#        nspace,ndigs,nfill=numspace(dens)
#        row+=nspace*' '+str(dens)
#        # emission
#        emission='0.00'
#        nspace,ndigs,nfill=numspace(float(emission))
#        row+=nspace*' '+emission
#        row+='\n'
#        f.write(row)

## Plotting

### Vertical profile

In [ ]:
# Open figure 
fig = plt.figure(figsize=(22,12)) 

# define grid of plots
gs = gridspec.GridSpec(nrows=4, ncols=8)

# x,y specify the subplot
x=0
y=0

#loop over bins
for ibin in range(len(dat1)):
    
    #open subplots
    ax = fig.add_subplot(gs[y, x])

    # set bin name as title
    ax.set_title(binnames[ibin]+' $\mu m$')
    
    # draw grid
    ax.grid()

    # get profile data
    xpoints=[dat1[ibin],dat1[ibin],dat2[ibin],0]
    ypoints=[0.,1.5,3.8,plumeheight]
    
    x_p=xp_plt[ibin]
    y_p=yp_plt[ibin]
    
    # plot profile
    ax.plot(xpoints,ypoints)
    ax.plot(y_p,x_p)
    ax.set_xlim(0)
    ax.set_ylim(0)
    

    # increas x,y
    x+=1
    if x > 7:
        x = 0
        y+=1

 ### Cross Section through the concentration field

In [ ]:
# Open figure 
fig = plt.figure(figsize=(22,12))

# define grid of plots
gs = gridspec.GridSpec(nrows=4, ncols=8)

# x,y specify the subplot
x=0
y=0

#loop over bins
for ibin in range(len(dat1)):
    
    #open subplots
    ax = fig.add_subplot(gs[y, x])
    
    # set bin name as title
    ax.set_title(binnames[ibin]+' $\mu m$')
    
    # draw grid
    ax.grid()

    # get stored data
    contour=contour_plt[ibin]
    
    # masked the areas where contour <= 0. so that they appear white in the plot
    contour = ma.masked_where(contour <= 0.,contour)
    
    
    mid=int(ngridpoints/2-1)
    con = ax.contourf(X[0,:,:], Z[:,0,:], contour[:,mid,:],cmap='jet') #ppgb
    fig.colorbar(con)


    # increas x,y
    x+=1
    if x > 7:
        x = 0
        y+=1

### 3d scatter plot of the particles

In [ ]:
# Open figure 
fig = plt.figure(figsize=(22,12)) 

# define grid of plots
gs = gridspec.GridSpec(nrows=4, ncols=8)

# x,y specify the subplot
x=0
y=0

#loop over bins
for ibin in range(len(dat1)):
    
    #open subplots
    ax = fig.add_subplot(gs[y, x],projection='3d')
    
    # set bin name as title
    ax.set_title(binnames[ibin]+' $\mu m$')
    
    # draw grid
    ax.grid()

    


    # increas x,y
    x+=1
    if x > 7:
        x = 0
        y+=1